In [1]:
import pymongo
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

In [2]:
client = MongoClient()
db = client.companies
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

### Based on the information provided, we are going to select a possible location for the office based on the available information.

The applied filter assums a **number of emplooyees of 90**. 

The reason? Currently, a huge number of companies have implanted a ***work from home policy*** but, additionally the company ***expect to increase their employees number in the future***, so being continously changing the location would be counterproductive.

Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [3]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city":{"$ne": "",}}
project1 = {"_id":0,"offices.city": 1,"number_of_employees": 1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [4]:
pd.DataFrame.from_dict(results1)

,number_of_employees,offices
0,90,{'city': 'Amsterdam'}
1,90,{'city': 'San Francisco'}
2,90,{'city': 'Singapore'}
3,90,{'city': 'Denver'}
4,90,{'city': 'San Francisco'}
5,90,{'city': 'Palo Alto'}
6,90,{'city': 'Chicago'}
7,90,{'city': 'San Francisco'}
8,90,{'city': 'Irving'}
9,90,{'city': 'South San Francisco'}


Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [5]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city": {'$eq': "New York"}}
project1 = {"_id":0,"name":1,"offices.city": 1,"number_of_employees": 1,"offices.latitude":1,"offices.longitude":1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [6]:
office_location = list(results1)
office_location

[{'name': 'Aleri',
  'number_of_employees': 90,
  'offices': {'city': 'New York',
   'latitude': 40.7592189,
   'longitude': -73.9783534}}]

In [7]:
pd.DataFrame.from_dict(office_location)

,name,number_of_employees,offices
0,Aleri,90,"{'city': 'New York', 'latitude': 40.7592189, '..."


## OFFICE LOCATION

In [8]:
office_lat = 40.7592189
office_long = -73.9783534

In [9]:
office_data = [("office","40.7592189","-73.9783534","POINT (-73.9783534 40.7592189)","office location")]
df_office = pd.DataFrame(office_data, columns = ["name","latitud","longitud","geometry","criteria"]) 
df_office

,name,latitud,longitud,geometry,criteria
0,office,40.7592189,-73.9783534,POINT (-73.9783534 40.7592189),office location


In [10]:
map_1 = folium.Map(location = [office_lat, office_long], zoom_start = 16)
map_1

In [11]:
office_loc = folium.Marker(location = [office_lat, office_long], tooltip = "Office location proposal")
# tooltip nos sirve para crear texto en los marcadores
office_loc.add_to(map_1)
map_1

icono = Icon(color = "blue",
             prefix = "fa",
             icon = "building-o",
             icon_color = "black",
             tooltip = "Office location proposal")

In [12]:
office_location = [office_lat, office_long]
marker_office = Marker(location = office_location, icon = icono)
marker_office.add_to(map_1)
map_1

In [13]:
type(office_location)

list

In [14]:
load_dotenv()

True

In [15]:
city = "New York"
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [16]:
donde = "New York"
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()

In [17]:
print(data)

{'standard': {'addresst': {}, 'city': 'New York', 'prov': 'US', 'countryname': 'United States of America', 'postal': {}, 'confidence': '0.90'}, 'longt': '-73.95861', 'alt': {'loc': {'longt': '-73.9536869565217', 'prov': 'NY', 'city': 'NEW YORK', 'postal': '10075', 'score': '25', 'latt': '40.7724386956522'}}, 'elevation': {}, 'latt': '40.68908'}


In [18]:
nyc = geocode(city)

In [19]:
nyc

{'type': 'Point', 'coordinates': [-73.95861, 40.68908]}

In [20]:
nyc = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}

In [21]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

### Búsqueda de Starbucks

In [22]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [23]:
parametros = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"Starbucks",
              "radius":500
}

In [24]:
resp = requests.get(url= url_query, params=parametros)
data = json.loads(resp.text)

In [25]:
data.keys()

dict_keys(['meta', 'response'])

In [26]:
decoding_data = data.get("response")

In [27]:
decoded = decoding_data.get("groups")[0]

In [28]:
starbucks = decoded.get("items")

In [29]:
map_starbucks = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [30]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [31]:
print(getFromDict(starbucks[0],map_starbucks))

Starbucks


In [32]:
starbucks_nyc = []
for dic in starbucks:
    paralista = {}
    paralista["name"] = getFromDict(dic,map_starbucks)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    starbucks_nyc.append(paralista)

In [33]:
starbucks_nyc[0]

{'name': 'Starbucks',
 'latitud': 40.75814852475969,
 'longitud': -73.97941981951507}

In [34]:
df_starbucks = pd.DataFrame(starbucks_nyc)
df_starbucks.head()

,name,latitud,longitud
0,Starbucks,40.758149,-73.979420
1,Starbucks,40.759535,-73.980981
2,Starbucks,40.760602,-73.976467
3,Starbucks,40.761845,-73.978700
4,Starbucks,40.756665,-73.974397


In [35]:
gdf_starbucks = gpd.GeoDataFrame(df_starbucks, geometry = gpd.points_from_xy(df_starbucks.longitud,df_starbucks.latitud))
gdf_starbucks.head()

,name,latitud,longitud,geometry
0,Starbucks,40.758149,-73.979420,POINT (-73.97942 40.75815)
1,Starbucks,40.759535,-73.980981,POINT (-73.98098 40.75953)
2,Starbucks,40.760602,-73.976467,POINT (-73.97647 40.76060)
3,Starbucks,40.761845,-73.978700,POINT (-73.97870 40.76185)
4,Starbucks,40.756665,-73.974397,POINT (-73.97440 40.75666)


In [36]:
gdf_starbucks = gdf_starbucks.assign(name = [ "Starbucks"]*20,
               criteria = ["Starbucks"]*20)
gdf_starbucks.head()

,name,latitud,longitud,geometry,criteria
0,Starbucks,40.758149,-73.979420,POINT (-73.97942 40.75815),Starbucks
1,Starbucks,40.759535,-73.980981,POINT (-73.98098 40.75953),Starbucks
2,Starbucks,40.760602,-73.976467,POINT (-73.97647 40.76060),Starbucks
3,Starbucks,40.761845,-73.978700,POINT (-73.97870 40.76185),Starbucks
4,Starbucks,40.756665,-73.974397,POINT (-73.97440 40.75666),Starbucks


In [37]:
Map(Layer(gdf_starbucks, popup_hover = [popup_element("name","Starbucks in NYC")]))

### 2. Looking for vegan restaurants

In [38]:
url2 = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"

In [39]:
parametros_vegan = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"vegans",
                "radius":500
}

In [40]:
resp2 = requests.get(url = url_query, params = parametros_vegan)
data2 = json.loads(resp2.text)

In [41]:
data2.keys()

dict_keys(['meta', 'response'])

In [42]:
decoding_data2 = data2.get("response")

In [43]:
decoded2 = decoding_data2.get("groups")[0]

In [44]:
vegan = decoded2.get("items")

In [45]:
map_vegan = ["venue","name"]
m_latitudvegan = ["venue","location","lat"]
m_longitudvegan = ["venue","location","lng"]

In [46]:
def getFromDict(diccionario2,mapa2):
    return reduce (operator.getitem,mapa2,diccionario2)

In [47]:
print(getFromDict(vegan[0],map_vegan))

Beyond Sushi


In [48]:
vegan_nyc = []
for dic in vegan:
    paralista2 = {}
    paralista2["name"] = getFromDict(dic,map_vegan)
    paralista2["latitud"] = getFromDict(dic,m_latitudvegan)
    paralista2["longitud"] = getFromDict(dic,m_longitudvegan)
    vegan_nyc.append(paralista2)

In [49]:
vegan_nyc[0]

{'name': 'Beyond Sushi',
 'latitud': 40.76345369164551,
 'longitud': -73.9771627466244}

In [50]:
df_vegans = pd.DataFrame(vegan_nyc)
df_vegans.head()

,name,latitud,longitud
0,Beyond Sushi,40.763454,-73.977163
1,by CHLOE.,40.758063,-73.978854
2,Van Leeuwen Ice Cream,40.759578,-73.980338
3,Taco Dumbo,40.760295,-73.977430
4,L’Avenue,40.758756,-73.977673


In [51]:
df_vegans[:5]

,name,latitud,longitud
0,Beyond Sushi,40.763454,-73.977163
1,by CHLOE.,40.758063,-73.978854
2,Van Leeuwen Ice Cream,40.759578,-73.980338
3,Taco Dumbo,40.760295,-73.977430
4,L’Avenue,40.758756,-73.977673


In [52]:
gdf_vegan = gpd.GeoDataFrame(df_vegans, geometry = gpd.points_from_xy(df_vegans.longitud,df_vegans.latitud))
gdf_vegan.head()

,name,latitud,longitud,geometry
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345)
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806)
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958)
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029)
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876)


In [53]:
gdf_vegan = gdf_vegan.assign(name = ["Beyond Sushi",
                                        "by CHLOE.",
                                        "Van Leeuwen Ice Cream",
                                        "Taco Dumbo",
                                        "L’Avenue",
                                        "Taco Dumbo",
                                        "Urbanspace W52",
                                        "The Halal Guys",
                                        "The Little Beet",
                                        "Lenwich by Lenny's",
                                        "Le Pain Quotidien",
                                        "Mysttik Masala",
                                        "Museum of Modern Art (MoMA)",
                                        "Fogo de Chão",
                                        "Aldo Sohm Wine Bar",
                                        "Gregorys Coffee",
                                        "Pret A Manger",
                                        "Devon & Blakely",
                                        "Fig & Olive",
                                        "The Modern",
                                        "Black Tap",
                                        "Simon Sips",
                                        "Barilla Restaurants",
                                        "Estiatorio Milos",
                                        "Le Pain Quotidien",
                                        "Cock & Bull British Pub and Eatery",
                                        "Europa Cafe",
                                        "Forty2West",
                                        "Butter Midtown",
                                        "Natureworks"],
                                criteria = ["vegan"]*30)
gdf_vegan.head()

,name,latitud,longitud,geometry,criteria
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345),vegan
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806),vegan
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958),vegan
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029),vegan
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876),vegan


In [54]:
Map(Layer(gdf_vegan, popup_hover = [popup_element("name","Vegan restaurants in NYC")]))

### 3. Looking for a basketball stadium


In [55]:
url3 = 'https://api.foursquare.com/v2/venues/explore'
basketball_stadium = "4bf58dd8d48988d18b941735"
# Ubicado al mens a 10 km

In [56]:
parametros_basket = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"basket_stadium",
                "radius":1000
}

In [57]:
resp3 = requests.get(url = url_query, params = parametros_basket)
data3 = json.loads(resp3.text)

In [58]:
data3.keys()

dict_keys(['meta', 'response'])

In [59]:
decoding_data3 = data3.get("response")

In [60]:
decoded3 = decoding_data3.get("groups")[0]

In [61]:
basket_stadium = decoded3.get("items")

In [62]:
map_stadium = ["venue","name"]

In [63]:
m_latitud_stadium = ["venue","location","lat"]
m_longitud_stadium = ["venue","location","lng"]
def getFromDict(diccionario3,mapa3):
    return reduce (operator.getitem,mapa3,diccionario3)

In [64]:
print(getFromDict(basket_stadium[0],map_stadium))

Regal E-Walk 4DX & RPX


In [65]:
basket_stadium_nyc = []
for dic in basket_stadium:
    paralista3 = {}
    paralista3["name"] = getFromDict(dic,map_stadium)
    paralista3["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista3["longitud"] = getFromDict(dic,m_longitud_stadium)
    basket_stadium_nyc.append(paralista3)

In [66]:
basket_stadium_nyc[0]
df_stadium = pd.DataFrame(basket_stadium_nyc)
df_stadium.head()

,name,latitud,longitud
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286
1,Zaro's Bakery,40.752875,-73.976946
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395


In [67]:
basketball_stadiums_location = list(df_stadium)
basketball_stadiums_location

['name', 'latitud', 'longitud']

In [68]:
gdf_stadiums = gpd.GeoDataFrame(df_stadium, geometry = gpd.points_from_xy(df_stadium.longitud,df_stadium.latitud))
gdf_stadiums

,name,latitud,longitud,geometry
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,POINT (-73.98929 40.75696)
1,Zaro's Bakery,40.752875,-73.976946,POINT (-73.97695 40.75288)
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,POINT (-73.98710 40.75784)
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,POINT (-73.98739 40.75721)


In [69]:
gdf_stadiums = gdf_stadiums.assign(name = [ "Regal E-Walk 4DX & RPX",
                           "Zaro's Bakery",
                           "Boomer Esiason's Stadium Grill",
                           "Stadium Grill At Bowlmor Lanes"],
               criteria = ["basketball stadium","basketball stadium","basketball stadium","basketball stadium"])
gdf_stadiums.head()

,name,latitud,longitud,geometry,criteria
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,POINT (-73.98929 40.75696),basketball stadium
1,Zaro's Bakery,40.752875,-73.976946,POINT (-73.97695 40.75288),basketball stadium
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,POINT (-73.98710 40.75784),basketball stadium
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,POINT (-73.98739 40.75721),basketball stadium


In [70]:
Map(Layer(gdf_stadiums, popup_hover = [popup_element("name","Basketball Stadium")]))

### 4. Night clubs

In [71]:
url4 = 'https://api.foursquare.com/v2/venues/explore'
night_clubs = "4bf58dd8d48988d11f941735"

In [72]:
parametros_night_clubs = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"night_clubs","radius":500
}

resp4 = requests.get(url = url_query, params = parametros_night_clubs)
data4 = json.loads(resp4.text)

In [73]:
data4.keys()

dict_keys(['meta', 'response'])

In [74]:
decoding_data4 = data4.get("response")

In [75]:
'''for k,v in decoding_data4.items():
    print(k,v,"\n")'''

'for k,v in decoding_data4.items():\n    print(k,v,"\n")'

In [76]:
decoded4 = decoding_data4.get("groups")[0]

In [77]:
night_clubs = decoded4.get("items")

In [78]:
map_night_clubs = ["venue","name"]

In [79]:
m_latitud_nightclubs = ["venue","location","lat"]
m_longitud_nightclubbs = ["venue","location","lng"]

In [80]:
def getFromDict(diccionario4,mapa4):
    return reduce (operator.getitem,mapa4,diccionario4)

In [81]:
print(getFromDict(night_clubs[0],map_night_clubs))

The Tonight Show starring Jimmy Fallon


In [82]:
night_clubs_nyc = []
for dic in night_clubs:
    paralista4 = {}
    paralista4["name"] = getFromDict(dic,map_night_clubs)
    paralista4["latitud"] = getFromDict(dic,m_latitud_nightclubs)
    paralista4["longitud"] = getFromDict(dic,m_longitud_nightclubbs)
    night_clubs_nyc.append(paralista4)

In [83]:
night_clubs_nyc[0]
df_night_clubs = pd.DataFrame(night_clubs_nyc)
df_night_clubs.head()

,name,latitud,longitud
0,The Tonight Show starring Jimmy Fallon,40.759362,-73.979807
1,Starry Night by Vincent van Gogh,40.761575,-73.977106
2,New York Sports Clubs,40.759151,-73.981686
3,Night Hotel Times Square,40.759131,-73.983739
4,Saturday Night Live Standby Line,40.758811,-73.979580


In [84]:
night_clubs_location = list(df_night_clubs)
night_clubs_location

['name', 'latitud', 'longitud']

In [85]:
gdf_night_clubs = gpd.GeoDataFrame(df_night_clubs, geometry = gpd.points_from_xy(df_night_clubs.longitud,df_night_clubs.latitud))
gdf_night_clubs.head()

,name,latitud,longitud,geometry
0,The Tonight Show starring Jimmy Fallon,40.759362,-73.979807,POINT (-73.97981 40.75936)
1,Starry Night by Vincent van Gogh,40.761575,-73.977106,POINT (-73.97711 40.76157)
2,New York Sports Clubs,40.759151,-73.981686,POINT (-73.98169 40.75915)
3,Night Hotel Times Square,40.759131,-73.983739,POINT (-73.98374 40.75913)
4,Saturday Night Live Standby Line,40.758811,-73.979580,POINT (-73.97958 40.75881)


In [86]:
gdf_night_clubs

,name,latitud,longitud,geometry
0,The Tonight Show starring Jimmy Fallon,40.759362,-73.979807,POINT (-73.97981 40.75936)
1,Starry Night by Vincent van Gogh,40.761575,-73.977106,POINT (-73.97711 40.76157)
2,New York Sports Clubs,40.759151,-73.981686,POINT (-73.98169 40.75915)
3,Night Hotel Times Square,40.759131,-73.983739,POINT (-73.98374 40.75913)
4,Saturday Night Live Standby Line,40.758811,-73.979580,POINT (-73.97958 40.75881)


In [87]:
gdf_night_clubs = gdf_night_clubs.assign(name = [ "The Tonight Show starring Jimmy Fallon",
                           "Starry Night by Vincent van Gogh",
                           "New York Sports Clubs",
                           "Night Hotel Times Square",
                           "Saturday Night Live Standby Line"],
               criteria = ["night_clubs","night_clubs","night_clubs","night_clubs","night_clubs"])
gdf_night_clubs.head()

,name,latitud,longitud,geometry,criteria
0,The Tonight Show starring Jimmy Fallon,40.759362,-73.979807,POINT (-73.97981 40.75936),night_clubs
1,Starry Night by Vincent van Gogh,40.761575,-73.977106,POINT (-73.97711 40.76157),night_clubs
2,New York Sports Clubs,40.759151,-73.981686,POINT (-73.98169 40.75915),night_clubs
3,Night Hotel Times Square,40.759131,-73.983739,POINT (-73.98374 40.75913),night_clubs
4,Saturday Night Live Standby Line,40.758811,-73.979580,POINT (-73.97958 40.75881),night_clubs


In [88]:
Map(Layer(gdf_night_clubs, popup_hover = [popup_element("name","Night clubs")]))

### 5. Transport availability

In [89]:
url5 = 'https://api.foursquare.com/v2/venues/explore'
travel_transport = "4d4b7105d754a06379d81259"

In [90]:
parametros_travel = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"travel_transport",
                ## "radius":1000
                    }

In [91]:
resp5 = requests.get(url = url_query, params = parametros_travel)
data5 = json.loads(resp5.text)

In [92]:
data5.keys()

dict_keys(['meta', 'response'])

In [93]:
decoding_data5 = data5.get("response")

In [94]:
decoded5 = decoding_data5.get("groups")[0]

In [95]:
transport = decoded5.get("items")
map_transport = ["venue","name"]

In [96]:
m_latitud_transport = ["venue","location","lat"]
m_longitud_transport = ["venue","location","lng"]
def getFromDict(diccionario5,mapa5):
    return reduce (operator.getitem,mapa5,diccionario5)

In [97]:
print(getFromDict(transport[0],map_transport))

Theater Row - The Acorn


In [98]:
transport_nyc = []
for dic in transport:
    paralista5 = {}
    paralista5["name"] = getFromDict(dic,map_stadium)
    paralista5["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista5["longitud"] = getFromDict(dic,m_longitud_stadium)
    transport_nyc.append(paralista5)

In [99]:
transport_nyc[0]

{'name': 'Theater Row - The Acorn',
 'latitud': 40.7585514688034,
 'longitud': -73.9934189068339}

In [100]:
df_transports = pd.DataFrame(transport_nyc)
df_transports.head()

,name,latitud,longitud
0,Theater Row - The Acorn,40.758551,-73.993419
1,Solstice Travel Vacations,40.758305,-73.967880
2,The Travel Inn,40.760124,-73.996154
3,United Bus and Travel,40.713707,-73.991714
4,Double Happiness Travel,40.713707,-73.991714


In [101]:
gdf_transports = gpd.GeoDataFrame(df_transports, geometry = gpd.points_from_xy(df_transports.longitud,df_transports.latitud))
gdf_transports.head()

,name,latitud,longitud,geometry
0,Theater Row - The Acorn,40.758551,-73.993419,POINT (-73.99342 40.75855)
1,Solstice Travel Vacations,40.758305,-73.967880,POINT (-73.96788 40.75830)
2,The Travel Inn,40.760124,-73.996154,POINT (-73.99615 40.76012)
3,United Bus and Travel,40.713707,-73.991714,POINT (-73.99171 40.71371)
4,Double Happiness Travel,40.713707,-73.991714,POINT (-73.99171 40.71371)


In [102]:
gdf_transports = gdf_transports.assign(name = [ "Theater Row - The Acorn",
                           "Solstice Travel Vacations",
                           "The Travel Inn",
                           "United Bus and Travel",
                           "Double Happiness Travel",
                           "K International Transport Co., Inc",
                           "Delgado Travel",
                           "Active Transport ServicesInc.",
                           "Rivas Travel & Multi Services"],
               criteria = ["transports","transports","transports","transports","transports","transports","transports","transports","transports"])
gdf_transports.head()

,name,latitud,longitud,geometry,criteria
0,Theater Row - The Acorn,40.758551,-73.993419,POINT (-73.99342 40.75855),transports
1,Solstice Travel Vacations,40.758305,-73.967880,POINT (-73.96788 40.75830),transports
2,The Travel Inn,40.760124,-73.996154,POINT (-73.99615 40.76012),transports
3,United Bus and Travel,40.713707,-73.991714,POINT (-73.99171 40.71371),transports
4,Double Happiness Travel,40.713707,-73.991714,POINT (-73.99171 40.71371),transports


In [103]:
Map(Layer(gdf_transports, popup_hover = [popup_element("name","Transports")]))

## Finally, we concatenate our criterias' dataframes.

In [104]:
df_criterias = pd.concat([df_office,gdf_starbucks, gdf_vegan,gdf_stadiums,gdf_night_clubs,gdf_transports])
df_criterias.head()

,name,latitud,longitud,geometry,criteria
0,office,40.7592189,-73.9783534,POINT (-73.9783534 40.7592189),office location
0,Starbucks,40.758149,-73.979420,POINT (-73.97941981951507 40.75814852475969),Starbucks
1,Starbucks,40.759535,-73.980981,POINT (-73.98098084615312 40.75953488433791),Starbucks
2,Starbucks,40.760602,-73.976467,POINT (-73.97646651001223 40.76060249450284),Starbucks
3,Starbucks,40.761845,-73.978700,POINT (-73.9787 40.761845),Starbucks


In [105]:
df_criterias = df_criterias

In [106]:
df_criterias.criteria.unique()

array(['office location', 'Starbucks', 'vegan', 'basketball stadium',
       'night_clubs', 'transports'], dtype=object)

In [153]:
map_f = folium.Map (location = [40.7592189,-73.9783534],zoom_start=18)

In [152]:
for i,row in df_criterias.iterrows():
    name = {"location" : [row["latitud"],row["longitud"]],
            "tooltip" : row["name"]}
    
    if row["criteria"] == 'office location':
        icon = Icon(color = "black",
                    prefix = "fa",
                    icon = "building-o",
                    icon_color = "white")
    elif row["criteria"] == 'transports':
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "plane",
                    icon_color = "black")
    elif row["criteria"] == 'vegan':
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "cutlery",
                    icon_color = "black")
    
    elif row["criteria"] == 'basketball stadium':
        icon = Icon(color = "orange",
                    prefix = "fa",
                    icon = "futbol-o",
                    icon_color = "black")
    elif row["criteria"] == 'Starbucks':
        icon = Icon(color = "darkpurple",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "white")
    else: 
        icon = Icon(color = "black",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "white")
        
        Marker(**name,icon = icon).add_to(map_f)

map_f